In [1]:
import json
import os
import pdb
import pickle
import random
import time
from collections import OrderedDict
from Process_Data.Datasets.LmdbDataset import Hdf5DelectDataset
import Process_Data.constants as c
import h5py

import numpy as np
import torch
import torch._utils
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from kaldi_io import read_mat
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tqdm import tqdm

from Define_Model.SoftmaxLoss import AngleLinear, AdditiveMarginLinear
# from Define_Model.model import PairwiseDistance
from Process_Data.Datasets.KaldiDataset import ScriptTrainDataset, \
    ScriptTestDataset, ScriptValidDataset
from Process_Data.audio_processing import ConcateOrgInput, mvnormal, ConcateVarInput, read_WaveInt
from TrainAndTest.common_func import create_model, load_model_args, args_model, args_parse

In [2]:
def calculate_outputs_and_gradients(inputs, model, target_label_idx,
                                    internal_batch=10):
    """sumary_line
    
    Keyword arguments:
    internal_batch -- internal batch_size has little effect on saliency mapplings.
    Return: return_description
    """
    
    # do the pre-processing
    
    if internal_batch != 1:
        inputs = torch.cat(inputs)
        real_inputs = []
        for i in range(len(inputs)):
            if i % internal_batch == 0:
                real_inputs.append(inputs[i:(i+internal_batch)])
        
        inputs = real_inputs

    gradients = []
    for s in inputs:
        s = Variable(s.cuda(), requires_grad=True)

        output, _ = model(s)
        output = F.softmax(output, dim=1)

        if target_label_idx is None:
            target_label_idx = torch.argmax(output, 1).item()

        index = torch.ones((output.size()[0], 1)) * target_label_idx
        index = torch.tensor(index, dtype=torch.int64)
        if s.is_cuda:
            index = index.cuda()

        output = output.gather(1, index)
        
        if internal_batch != 1:
            output = output.sum()

        # clear grad
        model.zero_grad()
        output.backward()

        gradient = s.grad.detach()#.cpu()
        gradients.append(gradient)

    gradients = torch.cat(gradients)

    return gradients, target_label_idx


def padding_baselines(data, baselines):
    this_baselinses = []
    for the_data in baselines:
        while the_data.shape[1] < data.shape[-2]:
            the_data = np.concatenate([the_data, the_data], axis=1)
        
        the_data = torch.tensor(the_data[:, :data.shape[-2]]).float().unsqueeze(0)
        this_baselinses.append(the_data)
        
    return torch.cat(this_baselinses, dim=0)

In [3]:
lstm_path = '/home/yangwenhao/project/lstm_speaker_verification'
root_dir = '/home/yangwenhao/local/project/SpeakerVerification-pytorch'
gradient_path = root_dir + '/Data/gradient'

In [ ]:
cam_method = 'integrad2' #   fullgrad grad_cam_pp integrad layer_cam
model_path = 'ThinResNet18/Mean_batch128_k7_seblock_downk1_avg1_SAP2_em256_dp01_alpha0_none1_wd5e5_varesmix8/arcsoft_sgd_rop/vox2/wave_fb80_dist2/123456'
train_set = 'vox2'
epoch = 25
check_yaml = root_dir + '/Data/checkpoint/{}/model.2023.08.02.yaml'.format(model_path)